In [ ]:
#ライブラリの読み込み
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import japanize_matplotlib
from matplotlib.patches import Patch
from datetime import datetime
from copy import deepcopy


#自作関数の読み込み
import modules.form_to_dict as ftd
import modules.shift_calculation_engine as sce
import modules.shift_visualization as sv
import modules.required_employees_culculation as rec

In [ ]:
# フォームの回答結果から従業員名(list)と出勤可否(dict)を作成
employees_name, availablity_dict_origin = ftd.form_of_employees_availability_to_dict("data/availability_test.csv")

In [ ]:
# データのカラムは下で定義

data = {
    "4F":{
        "room": "East",
        "guests_num": 20,
        "time_start": "10:00",
        "time_end": "13:30"
        },
    "5F":{
        "room": "West",
        "guests_num": 20,
        "time_start": "13:00",
        "time_end": "17:00"
        },
    "6F":{
       "room": "Terrace",
       "guests_num": 20,
       "time_start": "19:30",
       "time_end": "22:30"
    }
    
}

def time_to_index(time_str):
    time = datetime.strptime(time_str, "%H:%M")
    return time.hour * 2 + (time.minute // 30)

time_slots_dict = {}
for floor, info in data.items():
    key = f"{floor}_{info['room']}"
    start_index = time_to_index(info['time_start'])
    end_index = time_to_index(info['time_end'])
    time_slot = [0] * 48
    for i in range(start_index, end_index):
        time_slot[i] = 1
    time_slots_dict[key] = time_slot

In [ ]:
required_df = rec.required_employees(data)

# 各フロアごとの従業員数
employee_4f = required_df.loc["4F"].to_list()
employee_5f = required_df.loc["5F"].to_list()
employee_6f = required_df.loc["6F"].to_list()

In [ ]:
required_df

In [ ]:
floors_dict = {"4F": employee_4f, "5F": employee_5f, "6F": employee_6f}
min_nonworking_hours = float('inf')
best_order = None

# 時間を30分ごとのスロットに変換する関数
def time_to_slot(time):
    return int(time * 2) - 14

# シフトにアサインされた時間を出勤不可に更新する関数
def update_availability(shifts, availability):
    for name, times in shifts.items():
        if name in availability:
            start_slot = time_to_slot(times[0])
            end_slot = time_to_slot(times[1])
            for i in range(start_slot, end_slot):
                availability[name][i] = 0
                
    return availability

# 全ての順列に対して処理を行う
for floor_order in itertools.permutations(floors_dict.keys()):
    
    availability_dict = deepcopy(availablity_dict_origin)
    
    total_nonworking_hours = 0

    for floor in floor_order:
        employee_data = floors_dict[floor]
        # シフトアサインの計算
        shifts = sce.shift_calculation_engine(employees_name, availability_dict, employee_data)

        # 勤務時間が2時間以下の従業員を削除
        shifts = {e: (start, end) for e, (start, end) in shifts.items() if end - start > 2}

        # 出勤可否を更新
        availability_dict = update_availability(shifts, availability_dict)
        

    # 全従業員の勤務時間の合計を計算
    total_nonworking_hours = sum(sum(shifts) for shifts in availability_dict.values())
    
    print(f'従業員のnonworking時間の合計: {total_nonworking_hours}')

    # 最小の勤務時間を更新
    if total_nonworking_hours < min_nonworking_hours:
        min_nonworking_hours = total_nonworking_hours
        best_order = floor_order

# 最適な順序とその時の非勤務時間の合計を出力
print(f'最適な順序: {best_order}, 非勤務時間の合計: {min_nonworking_hours}')

In [ ]:
availability_calc =  deepcopy(availablity_dict_origin)

# 変換後のデータ構造
shifts_by_floor = {}

for floor in best_order:
        employee_data = floors_dict[floor]
        # シフトアサインの計算
        shifts = sce.shift_calculation_engine(employees_name, availability_calc, employee_data)

        # 勤務時間が2時間以下の従業員を削除
        shifts = {e: (start, end) for e, (start, end) in shifts.items() if end - start > 2}
        
        shifts_by_floor[floor] = shifts

        # 出勤可否を更新
        availability_dict = update_availability(shifts, availability_dict)

shifts_by_floor

In [ ]:
# ガントチャートを描画する関数を定義
def plot_gantt_chart(shifts_by_floor, colors):
    # 従業員名を集めてソート
    all_employees = sorted(set(employee for shifts in shifts_by_floor.values() for employee in shifts))

    # ガントチャートを描画
    fig, gnt = plt.subplots(figsize=(15, 8))

    # ガントチャートの設定
    gnt.grid(True)
    gnt.set_yticks([15*i for i in range(len(all_employees))])
    gnt.set_yticklabels(all_employees)
    gnt.set_xticks([i*60 for i in range(24)])
    gnt.set_xticklabels([f'{i}:00' for i in range(24)])
    gnt.set_xlim(0, 24*60)
    gnt.set_ylim(0, 15*len(all_employees))

    # フロアごとの色設定
    floor_colors = colors

    # 各従業員のシフトをプロット
    for floor, shifts in shifts_by_floor.items():
        for i, employee in enumerate(all_employees):
            if employee in shifts:
                start, end = shifts[employee]
                gnt.broken_barh([(start*60, (end-start)*60)], (15*i, 10),
                                facecolors=floor_colors[floor])

    # 凡例を追加
    legend_elements = [Patch(facecolor=color, label=f'{floor} Floor') for floor, color in colors.items()]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.show()
    
# フロアごとの色
colors = {'4F': 'blue', '5F': 'green', '6F': 'red'}

# ガントチャートをプロット
plot_gantt_chart(shifts_by_floor, colors)

### 次やること
・1日に対する各会場への従業員割当は完了したので、その割り当ての効用を最大化するようにしたい。  
・4階、5階、6階について、現在はそれぞれを順番に計算してシフトを割り当てているので、必然的に4階のアサインが充実して、6階のアサインが乏しくなる。  
・4,5,6階のすべての順番(3!通り)でアルゴリズムを回し、出勤可否のデータに対して損失関数を設定。一番損失が小さいものをシフトの結果とできるような仕組みを作りたい。  
・従業員のアサイン可能役職分を回す。  
・最終的にどの従業員がどの会場に何の役職でアサインされているかを表現

## 宴会データのcsvファイルをjson化

In [ ]:
# 宴会婚礼情報のcsvファイルをjson化


parties = pd.read_csv("data/parties_data.csv",encoding='cp932')

# 階層構造に基づいてデータを整理
grouped = parties.groupby(['日程', '会場', 'time'])
structured_data = grouped.apply(lambda x: x.drop(['日程', '会場', 'time'], axis=1).to_dict(orient='records')).reset_index().rename(columns={0: '詳細'})

# JSON形式に変換してファイルに保存
json_data = structured_data.to_json(orient='records', force_ascii=False)
#with open('data.json', 'w', encoding='utf-8') as f:
    #f.write(json_data)

# 結果を確認
print(json_data)
structured_data.head(50)